In [4]:
import pandas as pd
import numpy as np

In [5]:
from tmdbv3api import TMDb
from tmdbv3api import Movie
import json
import requests

tmdb = TMDb()
tmdb_movie = Movie()
tmdb.api_key = '4606882be56455abf5485c21adf285fb'

Extracting movie data from Wikipedia - 2020

In [6]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2020"

dfs = pd.read_html(link, header=0)

if len(dfs) >= 6:
    df1 = dfs[2]
    df2 = dfs[3]
    df3 = dfs[4]
    df4 = dfs[5]

    # Combine DataFrames vertically
    df = pd.concat([df1, df2, df3, df4], ignore_index=True)

    # Define the get_genre function
    def get_genre(x):
        genres = []
        result = tmdb_movie.search(x)
        if len(result) > 0:
            movie_id = result[0].id
            response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id, tmdb.api_key))
            data_json = response.json()
            if data_json['genres']:
                genre_str = " " 
                for i in range(0, len(data_json['genres'])):
                    genres.append(data_json['genres'][i]['name'])
                return genre_str.join(genres)
        return np.NaN

    # Apply the get_genre function to the 'Title' column
    df['genres'] = df['Title'].map(lambda x: get_genre(str(x)))

    # Print the resulting DataFrame
    print(df)
else:
    print("Not enough DataFrames found on the page.")

             Opening  Opening.1                  Title   
0      J A N U A R Y          3             The Grudge  \
1      J A N U A R Y         10             Underwater   
2      J A N U A R Y         10            Like a Boss   
3      J A N U A R Y         10          Three Christs   
4      J A N U A R Y         10      Inherit the Viper   
..               ...        ...                    ...   
270  D E C E M B E R         25       We Can Be Heroes   
271  D E C E M B E R         25      News of the World   
272  D E C E M B E R         25  One Night in Miami...   
273  D E C E M B E R         25  Promising Young Woman   
274  D E C E M B E R         30      Pieces of a Woman   

                                    Production company   
0    Screen Gems / Stage 6 Films / Ghost House Pict...  \
1    20th Century Fox / TSG Entertainment / Chernin...   
2                   Paramount Pictures / Artists First   
3                                            IFC Films   
4       Barry

In [7]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Ref.,genres
0,J A N U A R Y,3,The Grudge,Screen Gems / Stage 6 Films / Ghost House Pict...,Nicolas Pesce (director/screenplay); Andrea Ri...,[2],NaN,Horror Mystery
1,J A N U A R Y,10,Underwater,20th Century Fox / TSG Entertainment / Chernin...,"William Eubank (director); Brian Duffield, Ada...",[3],NaN,Horror Science Fiction Action Adventure
2,J A N U A R Y,10,Like a Boss,Paramount Pictures / Artists First,"Miguel Arteta (director); Sam Pitman, Adam Col...",[4],NaN,Comedy
3,J A N U A R Y,10,Three Christs,IFC Films,Jon Avnet (director/screenplay); Eric Nazarian...,NaN,NaN,Drama
4,J A N U A R Y,10,Inherit the Viper,Barry Films / Tycor International Film Company,Anthony Jerjen (director); Andrew Crabtree (sc...,[5],NaN,Crime Thriller Drama
...,...,...,...,...,...,...,...,...
270,D E C E M B E R,25,We Can Be Heroes,Netflix / Troublemaker Studios / Double R Prod...,Robert Rodriguez (director/screenplay); Priyan...,NaN,[241],Family Action Fantasy Comedy
271,D E C E M B E R,25,News of the World,Universal Pictures / Perfect World Pictures,Paul Greengrass (director/screenplay); Luke Da...,NaN,[242],Drama Western Adventure Action
272,D E C E M B E R,25,One Night in Miami...,Amazon Studios,Regina King (director); Kemp Powers (screenpla...,NaN,[243],Drama
273,D E C E M B E R,25,Promising Young Woman,Focus Features / FilmNation Entertainment,Emerald Fennell (director/screenplay); Carey M...,NaN,[244],Thriller Crime Drama


In [8]:
df_2020 = df[['Title','Cast and crew','genres']]
df_2020

,Title,Cast and crew,genres
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...,Horror Mystery
1,Underwater,"William Eubank (director); Brian Duffield, Ada...",Horror Science Fiction Action Adventure
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col...",Comedy
3,Three Christs,Jon Avnet (director/screenplay); Eric Nazarian...,Drama
4,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...,Crime Thriller Drama
...,...,...,...
270,We Can Be Heroes,Robert Rodriguez (director/screenplay); Priyan...,Family Action Fantasy Comedy
271,News of the World,Paul Greengrass (director/screenplay); Luke Da...,Drama Western Adventure Action
272,One Night in Miami...,Regina King (director); Kemp Powers (screenpla...,Drama
273,Promising Young Woman,Emerald Fennell (director/screenplay); Carey M...,Thriller Crime Drama


In [9]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

df_2020['director_name'] = df_2020['Cast and crew'].map(lambda x: get_director(x))


C:\Users\user\AppData\Local\Temp\ipykernel_4460\204755116.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020['director_name'] = df_2020['Cast and crew'].map(lambda x: get_director(x))


In [10]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

df_2020['actor_1_name'] = df_2020['Cast and crew'].map(lambda x: get_actor1(x))

def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

df_2020['actor_2_name'] = df_2020['Cast and crew'].map(lambda x: get_actor2(x))

def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

df_2020['actor_3_name'] = df_2020['Cast and crew'].map(lambda x: get_actor3(x))


C:\Users\user\AppData\Local\Temp\ipykernel_4460\1182333755.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020['actor_1_name'] = df_2020['Cast and crew'].map(lambda x: get_actor1(x))
C:\Users\user\AppData\Local\Temp\ipykernel_4460\1182333755.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020['actor_2_name'] = df_2020['Cast and crew'].map(lambda x: get_actor2(x))
C:\Users\user\AppData\Local\Temp\ipykernel_4460\1182333755.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy 

In [11]:
df_2020

,Title,Cast and crew,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...,Horror Mystery,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho
1,Underwater,"William Eubank (director); Brian Duffield, Ada...",Horror Science Fiction Action Adventure,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col...",Comedy,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek
3,Three Christs,Jon Avnet (director/screenplay); Eric Nazarian...,Drama,Jon Avnet,Richard Gere,Peter Dinklage,Walton Goggins
4,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...,Crime Thriller Drama,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs
...,...,...,...,...,...,...,...
270,We Can Be Heroes,Robert Rodriguez (director/screenplay); Priyan...,Family Action Fantasy Comedy,Robert Rodriguez,Priyanka Chopra Jonas,Pedro Pascal,YaYa Gosselin
271,News of the World,Paul Greengrass (director/screenplay); Luke Da...,Drama Western Adventure Action,Paul Greengrass,Tom Hanks,Helena Zengel,NaN
272,One Night in Miami...,Regina King (director); Kemp Powers (screenpla...,Drama,Regina King,Kingsley Ben-Adir,Eli Goree,Aldis Hodge
273,Promising Young Woman,Emerald Fennell (director/screenplay); Carey M...,Thriller Crime Drama,Emerald Fennell,Carey Mulligan,Bo Burnham,Alison Brie


In [12]:
df_2020 = df_2020.rename(columns={'Title':'movie_title'})

In [13]:
new_df20 = df_2020.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

new_df20

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho,Horror Mystery,The Grudge
1,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick,Horror Science Fiction Action Adventure,Underwater
2,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek,Comedy,Like a Boss
3,Jon Avnet,Richard Gere,Peter Dinklage,Walton Goggins,Drama,Three Christs
4,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs,Crime Thriller Drama,Inherit the Viper
...,...,...,...,...,...,...
270,Robert Rodriguez,Priyanka Chopra Jonas,Pedro Pascal,YaYa Gosselin,Family Action Fantasy Comedy,We Can Be Heroes
271,Paul Greengrass,Tom Hanks,Helena Zengel,NaN,Drama Western Adventure Action,News of the World
272,Regina King,Kingsley Ben-Adir,Eli Goree,Aldis Hodge,Drama,One Night in Miami...
273,Emerald Fennell,Carey Mulligan,Bo Burnham,Alison Brie,Thriller Crime Drama,Promising Young Woman


In [18]:
# Fill NaN values with 'unknown'

new_df20['actor_2_name'] = new_df20['actor_2_name'].replace(np.nan, 'unknown')
new_df20['actor_3_name'] = new_df20['actor_3_name'].replace(np.nan, 'unknown')

new_df20['movie_title'] = new_df20['movie_title'].str.lower()

new_df20['comb'] = new_df20['actor_1_name'] + ' ' + new_df20['actor_2_name'] + ' '+ new_df20['actor_3_name'] + ' '+ new_df20['director_name'] +' ' + new_df20['genres']

new_df20

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho,Horror Mystery,the grudge,Andrea Riseborough Demián Bichir John Cho Nico...
1,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick,Horror Science Fiction Action Adventure,underwater,Kristen Stewart Vincent Cassel Jessica Henwick...
2,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek,Comedy,like a boss,Tiffany Haddish Rose Byrne Salma Hayek Miguel ...
3,Jon Avnet,Richard Gere,Peter Dinklage,Walton Goggins,Drama,three christs,Richard Gere Peter Dinklage Walton Goggins Jon...
4,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs,Crime Thriller Drama,inherit the viper,Josh Hartnett Margarita Levieva Chandler Riggs...
...,...,...,...,...,...,...,...
270,Robert Rodriguez,Priyanka Chopra Jonas,Pedro Pascal,YaYa Gosselin,Family Action Fantasy Comedy,we can be heroes,Priyanka Chopra Jonas Pedro Pascal YaYa Gossel...
271,Paul Greengrass,Tom Hanks,Helena Zengel,unknown,Drama Western Adventure Action,news of the world,Tom Hanks Helena Zengel unknown Paul Greengras...
272,Regina King,Kingsley Ben-Adir,Eli Goree,Aldis Hodge,Drama,one night in miami...,Kingsley Ben-Adir Eli Goree Aldis Hodge Regina...
273,Emerald Fennell,Carey Mulligan,Bo Burnham,Alison Brie,Thriller Crime Drama,promising young woman,Carey Mulligan Bo Burnham Alison Brie Emerald ...


In [19]:
new_df20.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64

In [20]:
# drop null valued rows
new_df20 = new_df20.dropna(how='any')
new_df20.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64

In [21]:
# convert to csv file
new_df20.to_csv('new_data2020.csv',index=False)